### Setup Freyberg pilot points to use regularization and see what happens...

With our first attempt at pilot points, we saw shitty results.  So lets see if we can fix the problem by using regularization

In [1]:
%matplotlib inline
import os, shutil
import sys
sys.path.append("..")
import numpy as np
from IPython.display import Image
import pandas as pd
import matplotlib.pyplot as plt

import flopy as flopy
import pyemu

In [2]:
import freyberg_setup
freyberg_setup.setup_pest_pp()
working_dir = freyberg_setup.WORKING_DIR_PP
pst_name = freyberg_setup.PST_NAME_PP

['.DS_Store', 'botm.ref', 'extract_zone_array.py', 'forecasts_true.csv', 'freyberg.bas', 'freyberg.dbf', 'freyberg.dis', 'freyberg.hds', 'freyberg.heads', 'freyberg.heads_potobs.ins', 'freyberg.hyd', 'freyberg.list', 'freyberg.locations', 'freyberg.mpbas', 'freyberg.mpenpt', 'freyberg.mplist', 'freyberg.mpnam', 'freyberg.mppthln', 'freyberg.mpsim', 'freyberg.oc', 'freyberg.pcg', 'freyberg.rivflux', 'freyberg.shp', 'freyberg.shx', 'freyberg.travel', 'freyberg.truth.lpf', 'freyberg.truth.nam', 'freyberg.truth.rch', 'freyberg.truth.riv', 'freyberg.truth.wel', 'hk.truth.ref', 'hk.zones', 'ibound.ref', 'kzone.ref', 'mpath.in', 'potobs_group.csv', 'Process_output.py', 'really_noisy_heads.txt', 'run_true_model.py', 'strt.ref', 'Weights_and_best_PHI.xlsx']

changing model workspace...
   freyberg_pp
FloPy is using the following executable to run the model: /Users/jwhite/Dev/GW1876/activities/freyberg_pilot_points_2/freyberg_pp/mfnwt

                                  MODFLOW-NWT-SWR1 
    U.S.

/Users/jwhite/Dev/python/pyemu/pyemu/pst/pst_utils.py:419: UserWarning: pst_from_io_files has moved to pyemu.helpers and is also now avaiable as a Pst class method (Pst.from_io_files())
  "now avaiable as a Pst class method (Pst.from_io_files())")


run():./inschek freyberg.travel.ins freyberg.travel
run():./inschek vol.dat.ins vol.dat
starting interp point loop for 800 points
took 2.674241
run():./pestchek freyberg_pp.pst
run():./pestpp freyberg_pp.init.pst


In [11]:
pst = pyemu.Pst(os.path.join(working_dir,pst_name))

In the ``intro_to_regularization``, we talked about two common forms of regularization.  Here we will add both types to the control file.  

Now let's add a few preferred value equations for the recharge and well flux parameters:

In [12]:
pst.parameter_data.pargp.unique()

array(['rch', 'hk', 'w0', 'w1'], dtype=object)

In [13]:
pyemu.helpers.zero_order_tikhonov(pst,par_groups=["rch","w0","w1"])

In [14]:
pst.prior_information

,equation,obgnme,pilbl,weight
pilbl,,,,
rch_0,1.0 * log(rch_0) = 0.000000E+00,regulrch,rch_0,4.507576
rch_1,1.0 * log(rch_1) = 0.000000E+00,regulrch,rch_1,4.507576
w0_r09_c16,1.0 * log(w0_r09_c16) = 0.000000E+00,regulw0,w0_r09_c16,4.507576
w0_r11_c13,1.0 * log(w0_r11_c13) = 0.000000E+00,regulw0,w0_r11_c13,4.507576
w0_r20_c14,1.0 * log(w0_r20_c14) = 0.000000E+00,regulw0,w0_r20_c14,4.507576
w0_r26_c10,1.0 * log(w0_r26_c10) = 0.000000E+00,regulw0,w0_r26_c10,4.507576
w0_r29_c06,1.0 * log(w0_r29_c06) = 0.000000E+00,regulw0,w0_r29_c06,4.507576
w0_r34_c12,1.0 * log(w0_r34_c12) = 0.000000E+00,regulw0,w0_r34_c12,4.507576
w1_r09_c16,1.0 * log(w1_r09_c16) = 0.000000E+00,regulw1,w1_r09_c16,4.507576


Now, let's add preferred differnce regularization to the spatially distributed parameters

In [16]:
v = pyemu.geostats.ExpVario(contribution=1.0,a=2500.0)
gs = pyemu.geostats.GeoStruct(variograms=v,nugget=0.0)
df_pp = pyemu.gw_utils.pp_tpl_to_dataframe(os.path.join(working_dir,"hkpp.dat.tpl"))
cov = gs.covariance_matrix(df_pp.x,df_pp.y,df_pp.parnme)
pyemu.helpers.first_order_pearson_tikhonov(pst,cov,reset=False)
pst.prior_information

,equation,obgnme,pilbl,weight
pilbl,,,,
rch_0,1.0 * log(rch_0) = 0.000000E+00,regulrch,rch_0,4.507576
rch_1,1.0 * log(rch_1) = 0.000000E+00,regulrch,rch_1,4.507576
w0_r09_c16,1.0 * log(w0_r09_c16) = 0.000000E+00,regulw0,w0_r09_c16,4.507576
w0_r11_c13,1.0 * log(w0_r11_c13) = 0.000000E+00,regulw0,w0_r11_c13,4.507576
w0_r20_c14,1.0 * log(w0_r20_c14) = 0.000000E+00,regulw0,w0_r20_c14,4.507576
w0_r26_c10,1.0 * log(w0_r26_c10) = 0.000000E+00,regulw0,w0_r26_c10,4.507576
w0_r29_c06,1.0 * log(w0_r29_c06) = 0.000000E+00,regulw0,w0_r29_c06,4.507576
w0_r34_c12,1.0 * log(w0_r34_c12) = 0.000000E+00,regulw0,w0_r34_c12,4.507576
w1_r09_c16,1.0 * log(w1_r09_c16) = 0.000000E+00,regulw1,w1_r09_c16,4.507576


Now we need to adjust the inputs to pest the control how regularization is enforced.  The big knob is ``phimlim``.  But first, we need to change the estimation mode to "regularization"

In [23]:
pst.control_data.pestmode = "regularization"
pst.reg_data.phimlim

14

That's no good - way too low.  Theoritically, it should be the number of non-zero weighted obs, so let's start there:

In [22]:
pst.reg_data.phimlim = pst.nnz_obs
pst.reg_data.phimaccept = pst.nnz_obs * 1.1

In [24]:
pst.write(os.path.join(working_dir,pst_name))

In [ ]:
os.chdir(working_dir)
pyemu.helpers.start_slaves('.',"pestpp",pst_name,num_slaves=10,master_dir='.')
os.chdir("..")

master:pestpp freyberg_pp.pst /h :4004 in .
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_0
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_1
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_2
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_3
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_4
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_5
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_6
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_7
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_8
slave:pestpp freyberg_pp.pst /h localhost:4004 in ../slave_9
